# DeOldification process 
based on the FastAI colab found in https://github.com/jantic/DeOldify

In [0]:
import torch

if not torch.cuda.is_available():
    print('GPU not available.')

In [0]:
%cd /content
!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 2057 (delta 29), reused 32 (delta 17), pack-reused 2003
Receiving objects: 100% (2057/2057), 69.36 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (907/907), done.
/content/DeblurGAN/DeOldify


In [0]:
!pip install -r colab_requirements.txt


In [0]:
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True

In [0]:
# create a repo to store the models and download the pretrained model (Artistic version)

!mkdir 'models'
!wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

--2020-04-13 13:41:43--  https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth [following]
--2020-04-13 13:41:43--  https://www.dropbox.com/s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc2c1badc674b11fe3645dcea64.dl.dropboxusercontent.com/cd/0/inline/A1x7zLMhCtuIK9PGOXXL6spDLCylozLStDSII5L4Qe0HPMrdD8068x7pXOkT6dqqrB16Dt-MqhdZmreYidCjR01fwMrfo0KPgln122mFpdDnpg/file# [following]
--2020-04-13 13:41:43--  https://ucc2c1badc674b11fe3645dcea64.dl.dropboxusercontent.com/cd/0/inline/A1x7zLMhCtuIK9PGOXXL6spDLCylozLStDSII5L4Qe0HPMrdD8068x7pXOkT6dqqrB16Dt-MqhdZmreYidCjR01fwMr

In [0]:
# download a picture to test the model

!wget https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png -O ./resource_images/watermark.png

--2020-04-13 13:41:53--  https://media.githubusercontent.com/media/jantic/DeOldify/master/resource_images/watermark.png
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9210 (9.0K) [image/png]
Saving to: ‘./resource_images/watermark.png’

./resource_images/w 100%[===================>]   8.99K  --.-KB/s    in 0s      

2020-04-13 13:41:53 (68.0 MB/s) - ‘./resource_images/watermark.png’ saved [9210/9210]



In [0]:
# I then applied the deoldification to 6 pictures I downloaded to the colab instance 
# and put in /content/DeOldify/input_images

colorizer = get_image_colorizer(artistic=True)
for i in range(1,7,1):
    colorizer.plot_transformed_image('input_images/photo'+str(i)+'.png', render_factor=38, display_render_factor=True, figsize=(16,16))

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:442: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


KeyboardInterrupt: ignored

In [0]:
# The Artistic version of the model gave results that were not really realistic 
# for example some piece of clothing were washed out, like ty-died which was weird
# so I downloaded the Stable version of the pretrained model 

!wget https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

In [0]:
# once again I applied the model to my pictures which gave much more sound results
import fastai
from deoldify.visualize import *

torch.backends.cudnn.benchmark = True
render_factor=30
vis = get_image_colorizer(render_factor=render_factor, artistic=False)
for i in range(1,7,1):  
  vis.plot_transformed_image('input_images/photo'+str(i)+'.png', render_factor=render_factor, figsize=(30,30))

By default, when executing the `plot_transformed_image` function, the results are saved in the result_images folder from the Deoldify repo (each image has the same name as the input image).


# Super-resolution process


Now we have our colorized picture, we can then improve the resolution and increase the sharpness afterwards.

##SRGAN-tensorflow - DOESN'T WORK

In [0]:
%cd /content
!git clone https://github.com/brade31919/SRGAN-tensorflow.git

In [0]:
%cd SRGAN-tensorflow

In [0]:
!wget https://drive.google.com/a/gapp.nthu.edu.tw/uc?id=0BxRIhBA0x8lHNDJFVjJEQnZtcmc -O ./SRGAN_pre-trained

The URL to download the weight of the pretrained model seems to be broken or the `wget` doesn't work in colab
 -> moving on to another library with a pretrained model

##image-super-resolution


In [0]:
%cd ..
!git clone https://github.com/idealo/image-super-resolution.git

In [0]:
%cd image-super-resolution
!python3 setup.py install

In [0]:
!mkdir super-res-images

In [0]:
from ISR.models import RDN, RRDN
import numpy as np
from PIL import Image

model = RDN(weights='noise-cancel' ) #noise-cancel psnr-small psnr-large

for i in range(1,7):
  img = Image.open('/content/DeOldify/result_images/photo'+str(i)+'.png')
  lr_img = np.array(img.convert('RGB'))
  #model = RRDN(weights='gans')
  sr_img = model.predict(lr_img, by_patch_of_size=256, batch_size=1)
  Image.fromarray(sr_img).save('/content/image-super-resolution/super-res-images/photo'+str(i)+'_resed_up.jpg')

#Debluring process

##DeblurGAN - CONFUSING RESULTS



In [0]:
%cd ..
!git clone https://github.com/KupynOrest/DeblurGAN.git

In [0]:
%cd /content/DeblurGAN

In [0]:
!pip3 install dominate ssim visdom

* The weights of the pre-trained model need to be downloaded from the google drive of the developer of the library and then put in the colab instance in the checkpoints directory.
* In `test.py`, line 10,replace
`from ssim import SSIM` by `from util.metrics import SSIM` as per https://github.com/KupynOrest/DeblurGAN/issues/20 



In [0]:
import os

!pip install -q torch torchvision feather-format kornia pyarrow Pillow wandb nbdev fastprogress --upgrade

!pip install -q git+https://github.com/fastai/fastcore --upgrade

os._exit(00)

     |████████████████████████████████| 143kB 6.1MB/s 
     |████████████████████████████████| 63.2MB 144kB/s 
     |████████████████████████████████| 2.1MB 37.0MB/s 
     |████████████████████████████████| 1.4MB 40.2MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 460kB 47.8MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 112kB 42.9MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: isr 2.2.0 has requirement tensorflow==2.0.0, but you'll have tensorflow 2.2.0rc2 which is incompatible.


In [0]:
!python3 test.py --dataroot /content/image-super-resolution/super-res-images --model test --dataset_mode single --learn_residual

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
dataroot: /content/image-super-resolution/super-res-images
dataset_mode: single
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: 256
gan_type: wgan-gp
gpu_ids: [0]
how_many: 5000
input_nc: 3
isTrain: False
learn_residual: True
loadSizeX: 640
loadSizeY: 360
max_dataset_size: inf
model: test
nThreads: 2
n_layers_D: 3
name: experiment_name
ndf: 64
ngf: 64
no_dropout: False
no_flip: False
norm: instance
ntest: inf
output_nc: 3
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
which_direction: AtoB
which_epoch: latest
which_model_netD: basic
which_model_netG: resnet_9blocks
-------------- End ----------------
Opt.nThreads =  1
dataset [SingleImageDataset] was created
CustomDatasetDataLoader
---------- Networks initialized -------------
ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))

The results doesn't make much sense -> switching to another library


##SRN-Deblur


In [0]:
%cd /content
!git clone https://github.com/jiangsutx/SRN-Deblur.git
%cd SRN-Deblur

/content
Cloning into 'SRN-Deblur'...
remote: Enumerating objects: 69, done.
remote: Total 69 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (69/69), done.
/content/SRN-Deblur


In [0]:
%cd checkpoints
!bash download_model.sh
!ls color
%cd ..

/content/SRN-Deblur/checkpoints
--2020-04-13 14:07:15--  http://www.xtao.website/shared_url/srndeblur_models.zip
Resolving www.xtao.website (www.xtao.website)... 167.88.114.232
Connecting to www.xtao.website (www.xtao.website)|167.88.114.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81205330 (77M) [application/zip]
Saving to: ‘srndeblur_models.zip’

srndeblur_models.zi 100%[===================>]  77.44M  1.30MB/s    in 39s     

2020-04-13 14:07:55 (1.99 MB/s) - ‘srndeblur_models.zip’ saved [81205330/81205330]

Archive:  srndeblur_models.zip
   creating: color/
  inflating: color/deblur.model-523000.data-00000-of-00001  
  inflating: color/deblur.model-523000.index  
  inflating: color/deblur.model-523000.meta  
   creating: gray/
  inflating: gray/deblur.model-523000.data-00000-of-00001  
  inflating: gray/deblur.model-523000.index  
  inflating: gray/deblur.model-523000.meta  
   creating: lstm/
  inflating: lstm/deblur.model-523000.data-00000-of-00001 

In [0]:
#need scipy version <=1.1.0 to use the imread fonction from this library
!pip uninstall scipy -y
!pip install scipy==1.1.0

Uninstalling scipy-1.4.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/scipy-1.4.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/scipy/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_install.py", line 755, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.6/dist-packages/pip/_internal/req/req_uninstall.py", line 431, in _allowed_to_proceed
    return ask('Pro

In [0]:
#tensorflow version is now 2 by default on colab, change it to 1.X
%tensorflow_version 1.x
!mkdir /content/SRN-Deblur/results
!python3 run_model.py --input_path=/content/image-super-resolution/super-res-images --output_path=/content/SRN-Deblur/results --gpu=0

mkdir: cannot create directory ‘/content/SRN-Deblur/results’: File exists
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0413 14:10:04.352232 140679143237504 module_wrapper.py:139] From /content/SRN-Deblur/models/model.py:270: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0413 14:10:04.353641 140679143237504 module_wrapper.py:139] From /content/SRN-Deblur/models/model.py:74: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.


W0413 14:10:04.354356 140679143237504 module_wrapper.py:139] From /content/SRN-Deblur/models/model.py:85: The name tf.image.resize_images is deprecated. Please use tf.im